In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [2]:
import numpy as np
import pandas as pd

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset['price_log'] = np.log(combined_dataset['SalePrice'])
combined_dataset = combined_dataset.drop(['SalePrice', 'Id'], axis=1)
combined_dataset.shape

(2917, 81)

In [6]:
from kaggle.house_prices import quantile_rating as QR

ratings = QR.calc_ratings(
    df=combined_dataset, 
    target_var='price_log', 
    rating_quantiles=QR.calc_quantiles(
        sample=combined_dataset['price_log'],
        probs=[0.25, 0.5, 0.75]
    ), 
    categ_vars=helpers.get_character_colnames(combined_dataset)
)

ratings.head(7)

,var,value,rating
0,Alley,Grvl,1.500000
1,Alley,Pave,2.536585
2,Alley,_none_,2.531822
3,BldgType,1Fam,2.564039
4,BldgType,2fmCon,1.580645
5,BldgType,Duplex,1.692308
6,BldgType,Twnhs,1.883721


In [7]:
default_rating = ratings[ratings['var'].isna()]['rating'].iloc[0]
default_rating

2.5

In [15]:
step1 = pd.melt(
    frame=combined_dataset[helpers.get_character_colnames(combined_dataset)], 
    var_name='var', 
    value_name='value'
)

step1.head()

,var,value
0,Alley,_none_
1,Alley,_none_
2,Alley,_none_
3,Alley,_none_
4,Alley,_none_


In [35]:
step2 = (
    step1
    .join(
        ratings.set_index(['var', 'value']), 
        on=['var', 'value'], 
        how='left'
    )
    .fillna({'rating': default_rating})
)

step2.head()

len(step2[step2['rating'].isna()])

,var,value,rating
0,Alley,_none_,2.531822
1,Alley,_none_,2.531822
2,Alley,_none_,2.531822
3,Alley,_none_,2.531822
4,Alley,_none_,2.531822


0

In [43]:
step3 = (
    step2
    [['var', 'value']]
    .groupby('var')
    #mutate(id = row_number()) %>%
    #spread(var, rating) %>%
    #select(-id)
)

step3.head(1)

,var,value
0,Alley,_none_
2917,BldgType,1Fam
5834,BsmtCond,TA
8751,BsmtExposure,No
11668,BsmtFinType1,GLQ
14585,BsmtFinType2,Unf
17502,BsmtQual,Gd
20419,CentralAir,Y
23336,Condition1,Norm
26253,Condition2,Norm


In [42]:
step3.pivot(columns='var', values='value')

AttributeError: Cannot access callable attribute 'pivot' of 'DataFrameGroupBy' objects, try using the 'apply' method